<a href="https://colab.research.google.com/github/Daegeun02/linux/blob/main/Service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Service란? 

Linux OS가 부팅되었을 때, 생성되면서 종료될 때까지 실행되는 Process및 설정 파일을 __service__ 라고 한다.

## Service 생성

Service는 ```/etc/systemd/system``` 경로에 존재한다. 이 경로에 있는 service 들은 systemd에 의해 관리되고 있는 service라는 뜻이다.

Ex>

```
nvim /etc/systemd/system/test.service
```

## Service 파일 속성

Service 파일은 [Unit], [Service], [Install] 의 색션으로 구성되어 있다. 
전부 다 알아보긴 힘드니까 필요한 것  몇 개만 알아보고, 조금씩 채워나가 보자!

### Unit

1.   Description
    * 해당 Service 유닛에 대한 설명
2.   Requires
    * 상위 의존성을 구성한다.
    * 이 목록에 명시된 유닛이 정상적으로 동작할 경우에만 이 유닛을 시작한다.
3.   RequiresOverridable
    * Requires와 유사하하다.
    * 상위 의존성 유닛 구동에 실패하더라도 이를 무시하고 시작한다.

4.   Requisite
    * Requires와 유사하다.
    * 상위 의존성 유닛이 시작되지 않으면, 바로 실패를 반환한다.

5.   Wants
    * Requires의 완화된 버전
    * 목록에 포함된 유닛이 시작되지 않아도, 시작된다.

6.   Conflicts
    * 베타적 관계를 설정한다.
    * 만일 '유닛1'의 "Conflicts=" 설정이 '유닛2'로 되어있다면, '유닛1'이 시작된 경우 '유닛2'가 중지되고, '유닛1'이 중지된 경우 '유닛2'가 시작한다. 이 옵션은 "After="와 "Before=" 옵션과는 독립적으로 작동한다.

7.   Before
    * 유닛 시작 전 관계를 설정한다.
    * Requires 설정과는 독립적이다.
    * 이 옵션에 나열된 유닛이 시작되기 전에 이 유닛이 시작된다.

8.   After
    * 유닛 시작 후 관계를 설정한다.
    * Requires 설정과는 독립적이다.
    * 이 옵션에 나열된 유닛이 시작하고나서 이 유닛이 시작된다.

9.   OnFailure
    * 해당 유닛이 실패상태가 되면 수행할 유닛 목록을 지정한다.


### Service

1. Type
    * 유닛의 타입을 선언한다.
    - 1) __simple__(default)
        * 유닛이 시작된 경우 systemd는 즉시 유닛의 시작이 완료되었다고 판단한다.
        * 다른 유닛과 통신하기 위해 소켓을 사용하는 경우 이 타입을 사용할 수 없다.

    - 2) __forking__
        * 자식 프로세스 생성이 완료되어야 systemd가 유닛의 시작이 완료되었다고 판단한다.
        * 부모 프로세스를 추적할 수 있도록 PIDFile 옵션을 설정해주어야 한다.

    - 3) __oneshot__
        * simple과 유사하지만 단입 작업을 수행하는데 적합한 타입이다.
        * 실행 후 해당 실행이 종료되더라도 RemainAfterExit=yes 설정을 통해 유닛을 활성와 상태로 간주할 수 있다.

    - 4) __notify__
        * simple과 동일하지만, 유닛이 시작되면 systemd에 시그널을 보낸다.
        * 시그널의 내용은 libsystemd.daemon.so에 선언되어 있다.

    - 5) __dbus__
        * D-Bus에 지정된 BusName이 준비될 때가지 대기한다.
        * D-Bus가 준비된 후에 윤시이 시작되었다고 간주한다.

2. Environment
    * 유닛에서 사용할 환경변수를 선언한다.
    * 반드시 Exec 옵션보다 상단에 위치해야 한다(당연!).

3. ExecStartPre
    * 서비스 시작하기 전에 실행할 명령을 설정

4. ExecStart
    * 시작 명령을 정의한다.
    * 실행 명령어는 반드시 절대 경로 또는 변수로 지정한다.
    * 다중 명령어를 지원한다.
    * Ex> 
    > ExecStart="command1" <br>
    > ExecStart="command2" <br>
    > ExecStart="command3" <br>
    or<br>
    > ExecStart="command1; command2; command3"

5. ExecStartPost
    * 서비스 종료 명령 이후에 실행할 명령을 설정

6. ExecStop
    * 중지 명령을 정의한다.
    * 사용법은 ExecStart와 같다.
    * 중지 방식은 KillMode 옵션에서 설정한다.

7. ExecStopPost
    * 서비스를 종료 명령 이후에 실행할 명령을 설정

8. ExecReload
    * 서비스가 reload 될때 필요한 명령어나 스크립트를 지정

9. KillMode
    * 중지 방법에 대해서 설정하는 옵션
    - 1. __control-group__(default)
        * 해당 유닛의 그룹까지 모두 중지시킨다.
    - 2. __process__
        * 해당 유닛만 중지시킨다.
    - 3. __none__
        * 아무 액션을 취하지 않는다.

10. RestartSec
    * 재시작 명령을 수행할때, 중지 이후 다시 시작하는데 대기하는 시간을 설정한다.
    * 기본값은 100ms
    * 각각 "min", "s", "ms" 단위로 설정된다.
    * Restart 옵션이 있는 경우에만 적용된다.

11. Restart
    - 1. __no__(default)
        * 유닛을 다시 시작하지 않는다.
    - 2. __on-success__
        * 유닛이 정상적으로 종료되었을 때만 재시작한다.
    - 3. __on-failure__
        * 유닛이 비정상적으로 종료되었을때 재시작한다.
        * 타임 아웃값 내 응답이 없는 경우 등등 일때 재시작 하게 된다.
    - 4. __on-watchdog__
        * "WatchdogSec="에 설정된 시간 내 응답이 없는 경우에만 재시작한다.
    - 5. __on-abort__
        * 지정되지 않은 리턴값을 받은 경우 재시작을 한다.
    - 6. __always__
        * 종료 상태 등과 무관하게 무조건 재시작한다.
        * 사용자가 중지해도 시스템이 다시 띄우게 된다.

12. WorkingDirectory
    * 유닛의 작업 디렉토리를 지정한다.
    * 별도의 지정이 없으면 root("/") 디렉토리를 사용한다.

### Install


부팅 시에 Unit이 활성화나 비활성화를 위해 사용하는 명령어이다. 값이 default.target이면 링크 파일을 생성하지 않고, multi-user.target 이면 링크 파일을 생성한다.

1. Alias
    * 유닛의 별칭을 지정한다.
    * systemctl enable 명령어에 별칭을 사용할 수 있다.
    * Alias는 유닛 파일의 확장자를 가지고 있어야 한다.
    (Ex> .service, .socket, .mount, .swap 등)

2. WantedBy
    * systemctl enable 명령어로 유닛을 등록할 때 필요한 유닛을 지정한다.
    * Unit 섹션의 Wants와 관련이 있다.

3. RequiredBy
    * systemctl enable 명령어로 유닛을 등록할 때 필요한 유닛을 지정한다.
    * Unit 섹션의 Requires와 관련이 있다.

4. Also
    * systemctl enable/disable 명령어로 유닛을 등록하거나 해제할 때 다른 유닛도 함께 등록/해제 할 수 있게 만드는 설정이다.

## SystemCtl

서비스 유닛 (.service)을 관리 및 제어하는 명령어이다. CentOS7 버전 부터 사용하는 명령어이며, 이전 버전에서는 Service 명령어로 사용이 가능하다. 

1. 시작
    * ```systemctl start [servicename]```

2. 종료
    * ```systemctl stop [servicename]```
3. 재시작
    * ```systemctl restart [servicename]```
4. 재시작 (시작된 서비스)
    * ```systemctl try-restart [servicename]```
5. 설정 Reload
    * ```systemctl reload [servicename]```
6. 상태 확인
    * ```systemctl status [servicename]```
7. 자동시작 활성화 (부팅시)
    * ```systemctl enable [servicename]```
8. 자동시작 비활성화 (부팅시)
    * ```systemctl disable [servicename]```

# Example

```
~ cd /etc/systemd/system
~ sudo nvim example.service
```

example.service (Shell file must be in ```/usr/local/bin``` directory)
```
[Unit]
Description=Example service

[Service]
Type=simple
WorkingDirectory=/usr/local/bin
ExecStart=example.sh

[Install]
WantedBy=default.target
```

```
~ sudo systemctl start [yourservicename]
~ sudo systemctl enable [yourservicename]
~ sudo reboot -p
```

example.sh과 같은 .sh 파일 작성에 대해서는 __shell.ipynb__ 참고하시면 됩니다.